In [2]:
#Importer vigtige pakker
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
import pandas_datareader.data as web

import datetime
import pydst
dst = pydst.Dst(lang='da')

In [3]:
# a. loading data
dst.get_variables('NAB10') # command to se 'id'
#dst.get_variables('NABP10').iloc[0]['values'] # command to see 'values' in 'TRANSAKT'

na10 = dst.get_data(table_id = 'NABP10', variables={'TRANSAKT':['B1GD','D1D'],'BRANCHE':['*'], 'PRISENHED':['*'], 'TID':['*']})
na10

,TRANSAKT,BRANCHE,PRISENHED,TID,INDHOLD
0,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1989,1101580
1,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1990,1123635
2,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1991,1136742
3,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1992,1156370
4,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1993,1162193
5,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1994,1215845
6,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1995,1254013
7,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1996,1286022
8,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1997,1326153
9,"B.1g Bruttoværditilvækst, BVT",I alt,"2010-priser, kædede værdier",1998,1351902


In [19]:
# a. loading data
dst.get_variables('NABB10') # command to se 'id'
#dst.get_variables('NABB10').iloc[0]['values'] # command to see 'values' in 'SOCIO'
nabb10 = dst.get_data(table_id = 'NABB10', variables={'SOCIO':['EMPH_DC','SALH_DC'],'BRANCHE':['*'],'TID':['*']})
nabb10

,SOCIO,BRANCHE,TID,INDHOLD
0,Samlede præsterede timer (1000 timer),I alt,1966,4604383
1,Samlede præsterede timer (1000 timer),I alt,1967,4537166
2,Samlede præsterede timer (1000 timer),I alt,1968,4467413
3,Samlede præsterede timer (1000 timer),I alt,1969,4453350
4,Samlede præsterede timer (1000 timer),I alt,1970,4449231
5,Samlede præsterede timer (1000 timer),I alt,1971,4361538
6,Samlede præsterede timer (1000 timer),I alt,1972,4304325
7,Samlede præsterede timer (1000 timer),I alt,1973,4248746
8,Samlede præsterede timer (1000 timer),I alt,1974,4174298
9,Samlede præsterede timer (1000 timer),I alt,1975,3951117
